# Оценка страхового возмещения с помощью нейронной сети


Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/allstate/train.csv")

In [ ]:
data

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,D,B,B,D,D,B,D,C,B,D,B,A,A,A,A,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,A,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,B,D,B,D,C,B,B,B,A,A,A,A,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,D,B,B,D,D,D,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,D,B,D,B,D,B,B,C,B,B,C,A,A,A,B,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,587620,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,B,A,A,A,D,B,C,B,D,B,B,C,B,B,B,A,A,A,A,A,D,D,D,E,E,A,P,I,D,A,A,D,E,J,G,B,BI,EG,A,G,BM,A,L,CK,0.347403,0.785784,0.613660,0.473202,0.939556,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,587624,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,B,A,C,B,B,B,A,A,A,A,A,D,D,D,E,C,A,P,K,A,A,A,E,E,H,F,B,BI,BT,A,AV,AU,E,J,DF,0.507661,0.555782,0.549770,0.802892,0.704268,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,587630,A,B,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,

In [ ]:
# Убираем id
data = data.drop(columns=['id'])

In [ ]:
data

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,...,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,...,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,D,E,E,A,P,I,D,A,A,D,E,J,G,B,BI,EG,A,G,BM,A,L,CK,0.347403,0.785784,0.613660,0.473202,0.939556,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,D,E,C,A,P,K,A,A,A,E,E,H,F,B,BI,BT,A,AV,AU,E,J,DF,0.507661,0.555782,0.549770,0.802892,0.704268,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,A,B,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,H,D,D,D,E,C,A,P,F,C,C,A,E,F,H,F,B,BI,DM,A,W,AF,A,K,DJ,0.484469,0.785784,0.792378,0.189137,0.482436,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
188316,A,B,A,A,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,C,D,C,E,C,D,T,L,J,A,A,K,G,F,J,K,BI,AI,A,AS,AJ,A,O,MD,0.438385,0.422197,0.298977,0.383428,0.340543,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87


In [ ]:
# Отделяем числа от букв
data_cat = data.iloc[:, 0:116]
data_cont = data.iloc[:, 116:131]

In [ ]:
data_cat

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,...,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116
0,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,B,D,D,B,D,C,B,D,B,A,A,A,A,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB
1,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,B,D,D,A,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP
2,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,...,D,B,B,B,D,B,D,C,B,B,B,A,A,A,A,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK
3,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,B,D,D,D,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ
4,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,D,B,D,B,B,C,B,B,C,A,A,A,B,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,C,B,D,B,B,C,B,B,B,A,A,A,A,A,D,D,D,E,E,A,P,I,D,A,A,D,E,J,G,B,BI,EG,A,G,BM,A,L,CK
188314,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,B,D,D,B,A,C,B,B,B,A,A,A,A,A,D,D,D,E,C,A,P,K,A,A,A,E,E,H,F,B,BI,BT,A,AV,AU,E,J,DF
188315,A,B,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,D,B,C,B,A,C,B,B,B,A,A,A,B,H,D,D,D,E,C,A,P,F,C,C,A,E,F,H,F,B,BI,DM,A,W,AF,A,K,DJ
188316,A,B,A,A,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,D,B,A,D,D,A,B,C,B,D,B,A,A,A,A,A,C,D,C,E,C,D,T,L,J,A,A,K,G,F,J,K,BI,AI,A,AS,AJ,A,O,MD


In [ ]:
data_cont

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,0.347403,0.785784,0.613660,0.473202,0.939556,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,0.507661,0.555782,0.549770,0.802892,0.704268,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,0.484469,0.785784,0.792378,0.189137,0.482436,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
188316,0.438385,0.422197,0.298977,0.383428,0.340543,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87


In [ ]:
# Каждому символу (или их сочетанию) сопоставляется число
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data_cat_encoded = data_cat.apply(le.fit_transform)


In [ ]:
data_cat_encoded

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,...,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116
0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,1,3,3,1,3,2,1,3,1,0,0,0,0,0,3,1,2,4,0,2,15,1,6,0,0,8,4,6,9,6,45,28,2,19,55,0,14,269
1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,1,3,3,0,1,2,1,3,1,0,0,0,0,0,3,3,2,4,4,3,15,11,5,0,0,4,4,8,10,10,33,65,0,22,38,0,14,85
2,0,1,0,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,3,1,1,1,3,1,3,2,1,1,1,0,0,0,0,0,3,3,2,4,4,0,1,11,14,0,1,4,5,7,5,0,2,85,0,28,5,0,8,153
3,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,1,3,3,3,1,2,1,3,1,0,0,0,0,0,3,3,2,4,4,3,15,8,3,0,0,4,4,8,10,10,33,67,2,39,4,0,14,79
4,0,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,3,1,3,1,1,2,1,1,2,0,0,0,1,5,3,1,3,4,4,0,12,5,9,0,0,3,4,10,6,1,66,50,2,50,38,0,10,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,2,1,3,1,1,2,1,1,1,0,0,0,0,0,3,3,3,4,4,0,12,8,3,0,0,3,4,9,6,1,33,106,0,32,38,0,11,55
188314,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,1,3,3,1,0,2,1,1,1,0,0,0,0,0,3,3,3,4,2,0,12,10,0,0,0,4,4,7,5,1,33,44,0,22,20,4,9,75
188315,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,3,1,2,1,0,2,1,1,1,0,0,0,1,5,3,3,3,4,2,0,12,5,2,2,0,4,5,7,5,1,33,87,0,48,5,0,10,79
188316,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,0,3,3,0,1,2,1,3,1,0,0,0,0,0,2,3,2,4,2,3,15,11,9,0,0,10,6,5,9,10,33,9,0,19,9,0,14,295


In [ ]:
data_cont

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,0.347403,0.785784,0.613660,0.473202,0.939556,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,0.507661,0.555782,0.549770,0.802892,0.704268,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,0.484469,0.785784,0.792378,0.189137,0.482436,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
188316,0.438385,0.422197,0.298977,0.383428,0.340543,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87


In [ ]:
# Воссоединяем, ранее разделённые данные
data_cat_encoded.reset_index(drop=True, inplace=True)
data_cont.reset_index(drop=True, inplace=True)

In [ ]:
data2 = pd.concat([data_cat_encoded, data_cont], axis = 1)

In [ ]:
data2

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,1,1,3,3,1,3,2,1,3,1,0,0,0,0,0,3,1,2,4,0,2,15,1,6,0,0,8,4,6,9,6,45,28,2,19,55,0,14,269,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1,3,3,0,1,2,1,3,1,0,0,0,0,0,3,3,2,4,4,3,15,11,5,0,0,4,4,8,10,10,33,65,0,22,38,0,14,85,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,0,1,0,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1,1,3,1,3,2,1,1,1,0,0,0,0,0,3,3,2,4,4,0,1,11,14,0,1,4,5,7,5,0,2,85,0,28,5,0,8,153,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,1,1,3,3,3,1,2,1,3,1,0,0,0,0,0,3,3,2,4,4,3,15,8,3,0,0,4,4,8,10,10,33,67,2,39,4,0,14,79,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,0,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,1,3,1,3,1,1,2,1,1,2,0,0,0,1,5,3,1,3,4,4,0,12,5,9,0,0,3,4,10,6,1,66,50,2,50,38,0,10,55,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,3,1,2,1,3,1,1,2,1,1,1,0,0,0,0,0,3,3,3,4,4,0,12,8,3,0,0,3,4,9,6,1,33,106,0,32,38,0,11,55,0.347403,0.785784,0.613660,0.473202,0.939556,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1,3,3,1,0,2,1,1,1,0,0,0,0,0,3,3,3,4,2,0,12,10,0,0,0,4,4,7,5,1,33,44,0,22,20,4,9,75,0.507661,0.555782,0.549770,0.802892,0.704268,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
# Среднее значение
mmean = data2.mean(axis=0)
# Стандартное отклонение
mstd = data2.std(axis=0)
data2  = data2 - mmean
data2 /= mstd
data2

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,-0.574802,1.143633,-0.240848,1.463781,-0.722439,-0.655725,-0.157776,-0.249597,1.226521,-0.418834,2.890349,-0.422227,-0.339544,-0.110631,-0.013438,-0.188699,-0.083664,-0.072586,-0.098457,-0.032931,-0.046882,-0.015113,2.258260,-0.186668,-0.328564,-0.251452,-0.345361,-0.201958,-0.142054,-0.138771,-0.170799,-0.08045,-0.071469,-0.055774,-0.03365,-0.452491,-0.369188,-0.334935,-0.163874,-0.213353,-0.19853,-0.095302,-0.151181,-0.300554,-0.153354,-0.068597,-0.061125,-0.037822,-0.226516,-0.607025,-0.081645,-0.221576,-0.2981,-0.157457,-0.027759,-0.031103,-0.127707,-0.035647,-0.039945,-0.048723,-0.062038,-0.01546,-0.020486,-0.0158,-0.110261,-0.215210,-0.06073,-0.02747,-0.040409,-0.011052,-0.232681,-0.769135,-0.469530,-0.138845,2.129897,-0.188308,0.06144,-0.024745,-0.343780,0.599156,0.448875,-0.156687,2.758342,0.396722,-0.04648,1.221871,-0.320404,-0.338597,-0.150755,-0.230187,-0.573062,-0.710736,0.462592,-1.511842,-0.764084,0.121602,-1.370707,0.506055,0.749216,-1.938721,0.899014,-0.224837,-0.566816,1.051490,-0.665439,-0.370787,0.920648,0.408493,1.126878,-1.556937,0.449649,-0.380008,1.224960,-0.542895,0.891654,1.578088,1.238747,-1.260932,-1.540467,1.409549,-0.848536,1.107905,-0.840068,-0.922089,1.023029,1.813213,0.363475,0.484635,1.547888,0.984891,-0.283792
1,-0.574802,1.143633,-0.240848,-0.683159,-0.722439,-0.655725,-0.157776,-0.249597,1.226521,2.387569,-0.345977,-0.422227,-0.339544,-0.110631,-0.013438,-0.188699,-0.083664,-0.072586,-0.098457,-0.032931,-0.046882,-0.015113,-0.442817,-0.186668,-0.328564,-0.251452,-0.345361,-0.201958,-0.142054,-0.138771,-0.170799,-0.08045,-0.071469,-0.055774,-0.03365,-0.452491,-0.369188,-0.334935,-0.163874,-0.213353,-0.19853,-0.095302,-0.151181,-0.300554,-0.153354,-0.068597,-0.061125,-0.037822,-0.226516,-0.607025,-0.081645,-0.221576,-0.2981,-0.157457,-0.027759,-0.031103,-0.127707,-0.035647,-0.039945,-0.048723,-0.062038,-0.01546,-0.020486,-0.0158,-0.110261,-0.215210,-0.06073,-0.02747,-0.040409,-0.011052,-0.232681,-0.769135,-0.469530,-0.138845,-0.469470,-0.188308,0.06144,-0.024745,-0.343780,0.599156,0.448875,-1.565603,-0.079072,0.396722,-0.04648,1.221871,-0.320404,-0.338597,-0.150755,-0.230187,-0.573062,-0.710736,0.462592,0.690767,-0.764084,0.121602,0.878977,1.184413,0.749216,1.064358,0.638047,-0.224837,-0.566816,-0.678551,-0.665439,0.767442,1.377035,1.507751,0.173802,-0.297037,-0.566973,-0.148451,0.348318,-0.542895,0.891654,-0.909695,-0.870535,1.109448,0.463931,0.578922,1.906005,-0.253456,-0.271141,0.573970,-0.738942,-0.316747,-0.739971,-0.605670,0.555949,-0.859468,-0.603886
2,-0.574802,1.143633,-0.240848,-0.683159,1.384192,-0.655725,-0.157776,-0.249597,1.226521,2.387569,2.890349,2.368384,2.945107,-0.110631,-0.013438,-0.188699,-0.083664,-0.072586,-0.098457,-0.032931,-0.046882,-0.015113,-0.442817,-0.186668,-0.328564,-0.251452,-0.345361,-0.201958,-0.142054,-0.138771,-0.170799,-0.08045,-0.071469,-0.055774,-0.03365,2.209976,-0.369188,-0.334935,-0.163874,-0.213353,-0.19853,-0.095302,-0.151181,-0.300554,-0.153354,-0.068597,-0.061125,-0.037822,-0.226516,-0.607025,-0.081645,-0.221576,-0.2981,-0.157457,-0.027759,-0.031103,-0.127707,-0.035647,-0.039945,-0.048723,-0.062038,-0.01546,-0.020486,-0.0158,-0.110261,-0.215210,-0.06073,-0.02747

In [ ]:
# Нужно удалить столбцы NaN - удаление вручную 1 столбца
#data2 = data2.drop(columns=['X11'])
#data2

In [ ]:
# Просмотр значений столбцов
pd.set_option('display.max_columns', 500)
data2.describe()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
count,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05
mean,-5.402405e-15,-2.264468e-16,-1.236509e-15,1.309893e-15,-2.338045e-15,4.353074e-15,6.054927e-15,1.612751e-15,-5.148437e-16,6.778004e-16,-4.915501e-16,-1.809942e-16,-7.232564e-15,1.980777e-15,-7.925047e-16,-1.662929e-15,3.250219e-15,-5.980293e-15,8.000066e-15,-9.992107e-16,-2.064617e-15,-1.366896e-15,6.001895e-16,6.436825e-15,-1.485750e-15,-4.913935e-15,1.536914e-15,-6.401046e-15,2.691948e-15,-4.671013e-15,-2.154219e-16,-2.039410e-15,-4.171758e-16,4.777285e-15,1.543481e-15,8.035248e-16,-1.483259e-16,-6.151006e-15,7.553246e-17,-1.559918e-16,2.191583e-15,-1.352105e-15,2.599419e-16,3.511316e-15,2.565404e-15,7.819480e-15,2.058546e-15,5.717485e-15,-3.800801e-15,-8.495336e-16,5.752357e-16,-2.032496e-15,-3.601611e-15,6.254552e-15,7.953795e-15,-5.529867e-16,-7.936690e-15,6.210425e-15,-7.618175e-16,2.220129e-15,-5.404539e-15,3.285238e-15,7.105278e-16,9.575817e-16,1.328900e-15,2.195716e-15,-4.257183e-16,2.395519e-15,2.929009e-15,5.083383e-15,-2.217753e-15,1.308935e-15,-7.776908e-16,-1.230545e-15,5.651521e-16,3.418590e-15,3.114348e-17,-2.076987e-15,7.825838e-16,-3.267364e-16,-3.774919e-15,3.682029e-15,7.164188e-15,-3.345342e-15,4.151695e-15,-2.237559e-15,-1.676200e-16,1.330264e-15,-3.393575e-15,2.240817e-15,1.691468e-15,8.632936e-16,-1.237542e-15,2.709907e-15,-1.374434e-15,-3.182421e-15,-1.789770e-16,-2.431595e-15,1.095119e-16,-1.268457e-15,1.380748e-15,5.076954e-16,2.705838e-15,-2.704511e-16,2.740765e-15,4.345999e-16,-2.052181e-16,-6.685969e-16,1.662296e-15,8.246524e-16,-2.1

In [ ]:
# .dropna - удаляет все столбцы, содержащие NaN
data2 = data2.dropna(axis='columns')
data2.describe()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
count,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05,1.883180e+05
mean,-5.402405e-15,-2.264468e-16,-1.236509e-15,1.309893e-15,-2.338045e-15,4.353074e-15,6.054927e-15,1.612751e-15,-5.148437e-16,6.778004e-16,-4.915501e-16,-1.809942e-16,-7.232564e-15,1.980777e-15,-7.925047e-16,-1.662929e-15,3.250219e-15,-5.980293e-15,8.000066e-15,-9.992107e-16,-2.064617e-15,-1.366896e-15,6.001895e-16,6.436825e-15,-1.485750e-15,-4.913935e-15,1.536914e-15,-6.401046e-15,2.691948e-15,-4.671013e-15,-2.154219e-16,-2.039410e-15,-4.171758e-16,4.777285e-15,1.543481e-15,8.035248e-16,-1.483259e-16,-6.151006e-15,7.553246e-17,-1.559918e-16,2.191583e-15,-1.352105e-15,2.599419e-16,3.511316e-15,2.565404e-15,7.819480e-15,2.058546e-15,5.717485e-15,-3.800801e-15,-8.495336e-16,5.752357e-16,-2.032496e-15,-3.601611e-15,6.254552e-15,7.953795e-15,-5.529867e-16,-7.936690e-15,6.210425e-15,-7.618175e-16,2.220129e-15,-5.404539e-15,3.285238e-15,7.105278e-16,9.575817e-16,1.328900e-15,2.195716e-15,-4.257183e-16,2.395519e-15,2.929009e-15,5.083383e-15,-2.217753e-15,1.308935e-15,-7.776908e-16,-1.230545e-15,5.651521e-16,3.418590e-15,3.114348e-17,-2.076987e-15,7.825838e-16,-3.267364e-16,-3.774919e-15,3.682029e-15,7.164188e-15,-3.345342e-15,4.151695e-15,-2.237559e-15,-1.676200e-16,1.330264e-15,-3.393575e-15,2.240817e-15,1.691468e-15,8.632936e-16,-1.237542e-15,2.709907e-15,-1.374434e-15,-3.182421e-15,-1.789770e-16,-2.431595e-15,1.095119e-16,-1.268457e-15,1.380748e-15,5.076954e-16,2.705838e-15,-2.704511e-16,2.740765e-15,4.345999e-16,-2.052181e-16,-6.685969e-16,1.662296e-15,8.246524e-16,-2.1

In [ ]:
xdata = data2

In [ ]:
ydata = data.iloc[:, 130]

In [ ]:
xdata

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,-0.574802,1.143633,-0.240848,1.463781,-0.722439,-0.655725,-0.157776,-0.249597,1.226521,-0.418834,2.890349,-0.422227,-0.339544,-0.110631,-0.013438,-0.188699,-0.083664,-0.072586,-0.098457,-0.032931,-0.046882,-0.015113,2.258260,-0.186668,-0.328564,-0.251452,-0.345361,-0.201958,-0.142054,-0.138771,-0.170799,-0.08045,-0.071469,-0.055774,-0.03365,-0.452491,-0.369188,-0.334935,-0.163874,-0.213353,-0.19853,-0.095302,-0.151181,-0.300554,-0.153354,-0.068597,-0.061125,-0.037822,-0.226516,-0.607025,-0.081645,-0.221576,-0.2981,-0.157457,-0.027759,-0.031103,-0.127707,-0.035647,-0.039945,-0.048723,-0.062038,-0.01546,-0.020486,-0.0158,-0.110261,-0.215210,-0.06073,-0.02747,-0.040409,-0.011052,-0.232681,-0.769135,-0.469530,-0.138845,2.129897,-0.188308,0.06144,-0.024745,-0.343780,0.599156,0.448875,-0.156687,2.758342,0.396722,-0.04648,1.221871,-0.320404,-0.338597,-0.150755,-0.230187,-0.573062,-0.710736,0.462592,-1.511842,-0.764084,0.121602,-1.370707,0.506055,0.749216,-1.938721,0.899014,-0.224837,-0.566816,1.051490,-0.665439,-0.370787,0.920648,0.408493,1.126878,-1.556937,0.449649,-0.380008,1.224960,-0.542895,0.891654,1.578088,1.238747,-1.260932,-1.540467,1.409549,-0.848536,1.107905,-0.840068,-0.922089,1.023029,1.813213,0.363475,0.484635,1.547888,0.984891,-0.283792
1,-0.574802,1.143633,-0.240848,-0.683159,-0.722439,-0.655725,-0.157776,-0.249597,1.226521,2.387569,-0.345977,-0.422227,-0.339544,-0.110631,-0.013438,-0.188699,-0.083664,-0.072586,-0.098457,-0.032931,-0.046882,-0.015113,-0.442817,-0.186668,-0.328564,-0.251452,-0.345361,-0.201958,-0.142054,-0.138771,-0.170799,-0.08045,-0.071469,-0.055774,-0.03365,-0.452491,-0.369188,-0.334935,-0.163874,-0.213353,-0.19853,-0.095302,-0.151181,-0.300554,-0.153354,-0.068597,-0.061125,-0.037822,-0.226516,-0.607025,-0.081645,-0.221576,-0.2981,-0.157457,-0.027759,-0.031103,-0.127707,-0.035647,-0.039945,-0.048723,-0.062038,-0.01546,-0.020486,-0.0158,-0.110261,-0.215210,-0.06073,-0.02747,-0.040409,-0.011052,-0.232681,-0.769135,-0.469530,-0.138845,-0.469470,-0.188308,0.06144,-0.024745,-0.343780,0.599156,0.448875,-1.565603,-0.079072,0.396722,-0.04648,1.221871,-0.320404,-0.338597,-0.150755,-0.230187,-0.573062,-0.710736,0.462592,0.690767,-0.764084,0.121602,0.878977,1.184413,0.749216,1.064358,0.638047,-0.224837,-0.566816,-0.678551,-0.665439,0.767442,1.377035,1.507751,0.173802,-0.297037,-0.566973,-0.148451,0.348318,-0.542895,0.891654,-0.909695,-0.870535,1.109448,0.463931,0.578922,1.906005,-0.253456,-0.271141,0.573970,-0.738942,-0.316747,-0.739971,-0.605670,0.555949,-0.859468,-0.603886
2,-0.574802,1.143633,-0.240848,-0.683159,1.384192,-0.655725,-0.157776,-0.249597,1.226521,2.387569,2.890349,2.368384,2.945107,-0.110631,-0.013438,-0.188699,-0.083664,-0.072586,-0.098457,-0.032931,-0.046882,-0.015113,-0.442817,-0.186668,-0.328564,-0.251452,-0.345361,-0.201958,-0.142054,-0.138771,-0.170799,-0.08045,-0.071469,-0.055774,-0.03365,2.209976,-0.369188,-0.334935,-0.163874,-0.213353,-0.19853,-0.095302,-0.151181,-0.300554,-0.153354,-0.068597,-0.061125,-0.037822,-0.226516,-0.607025,-0.081645,-0.221576,-0.2981,-0.157457,-0.027759,-0.031103,-0.127707,-0.035647,-0.039945,-0.048723,-0.062038,-0.01546,-0.020486,-0.0158,-0.110261,-0.215210,-0.06073,-0.02747

In [ ]:
ydata

0         2213.18
1         1283.60
2         3005.09
3          939.85
4         2763.85
           ...   
188313    1198.62
188314    1108.34
188315    5762.64
188316    1562.87
188317    4751.72
Name: loss, Length: 188318, dtype: float64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Перемешивание наборов
X_train, X_test, y_train, y_test = train_test_split(xdata, ydata, test_size=0.33, random_state=42)

In [ ]:
#from tensorflow.python.keras.datasets import boston_housing
# Импорт слоёв
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [ ]:
# Создание сети
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(1))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               16896     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 17,025
Trainable params: 17,025
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
def FindBatchSize(model):
    """#model: model architecture, that is yet to be trained"""
    import os, sys, psutil, gc, tensorflow, keras
    import numpy as np
    from keras import backend as K
    BatchFound= 16

    try:
        total_params= int(model.count_params());    GCPU= "CPU"
        #find whether gpu is available
        try:
            if K.tensorflow_backend._get_available_gpus()== []:
                GCPU= "CPU";    #CPU and Cuda9GPU
            else:
                GCPU= "GPU"
        except:
            from tensorflow.python.client import device_lib;    #Cuda8GPU
            def get_available_gpus():
                local_device_protos= device_lib.list_local_devices()
                return [x.name for x in local_device_protos if x.device_type == 'GPU']
            if "gpu" not in str(get_available_gpus()).lower():
                GCPU= "CPU"
            else:
                GCPU= "GPU"

        #decide batch size on the basis of GPU availability and model complexity
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <1000000):
            BatchFound= 64    
        if (os.cpu_count() <16) and (total_params <500000):
            BatchFound= 64  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <2000000) and (total_params >=1000000):
            BatchFound= 32      
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=2000000) and (total_params <10000000):
            BatchFound= 16  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=10000000):
            BatchFound= 8       
        if (os.cpu_count() <16) and (total_params >5000000):
            BatchFound= 8    
        if total_params >100000000:
            BatchFound= 1

    except:
        pass
    try:

        #find percentage of memory used
        memoryused= psutil.virtual_memory()
        memoryused= float(str(memoryused).replace(" ", "").split("percent=")[1].split(",")[0])
        if memoryused >75.0:
            BatchFound= 8
        if memoryused >85.0:
            BatchFound= 4
        if memoryused >90.0:
            BatchFound= 2
        if total_params >100000000:
            BatchFound= 1
        print("Batch Size:  "+ str(BatchFound));    gc.collect()
    except:
        pass

    memoryused= [];    total_params= [];    GCPU= "";
    del memoryused, total_params, GCPU;    gc.collect()
    return BatchFound

In [ ]:
FindBatchSize(model)

Batch Size:  64


64

In [ ]:
# Компиляция
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Обучение
# epochs -  показывает, сколько раз модель подвергается воздействию обучения. Эпоха – один проход вперёд или назад для всех примеров обучения.
# batch_size - количество обучающих примеров за одну итерацию. Чем больше batch size, тем больше места будет необходимо.
# validation_split - процент исходных данных, который будет использоваться для валидации выходных данных

model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.2, verbose=2) # epochs = 15 batch_size = 128

Epoch 1/20
789/789 - 2s - loss: 72443.3828 - mae: 191.0670 - val_loss: 64504.1172 - val_mae: 160.6082
Epoch 2/20
789/789 - 1s - loss: 46009.7227 - mae: 138.9445 - val_loss: 48074.2148 - val_mae: 121.5377
Epoch 3/20
789/789 - 1s - loss: 33416.4336 - mae: 106.9324 - val_loss: 38425.3125 - val_mae: 95.2628
Epoch 4/20
789/789 - 2s - loss: 26299.9688 - mae: 85.7019 - val_loss: 33426.0039 - val_mae: 78.2964
Epoch 5/20
789/789 - 1s - loss: 21933.1641 - mae: 71.1923 - val_loss: 29944.6172 - val_mae: 66.6186
Epoch 6/20
789/789 - 1s - loss: 19112.8008 - mae: 60.8934 - val_loss: 27080.3242 - val_mae: 57.1809
Epoch 7/20
789/789 - 1s - loss: 16987.7734 - mae: 53.6883 - val_loss: 25244.1816 - val_mae: 51.3607
Epoch 8/20
789/789 - 1s - loss: 15309.8232 - mae: 47.7589 - val_loss: 23928.4727 - val_mae: 46.9535
Epoch 9/20
789/789 - 1s - loss: 13948.0889 - mae: 43.4414 - val_loss: 22518.8848 - val_mae: 41.5826
Epoch 10/20
789/789 - 2s - loss: 12921.0088 - mae: 39.4425 - val_loss: 21850.8574 - val_mae: 38

In [ ]:
# Запуск предсказания
pred = model.predict(X_test)
print(pred)

[[1530.4155]
 [1046.0164]
 [5408.048 ]
 ...
 [3616.3762]
 [1504.6007]
 [ 822.326 ]]


In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
diff = np.array([])
for i in np.arange(0, len(y_test)):
  diff = np.append(diff, np.abs(pred[i][0] - y_test[i]))


In [ ]:
for i in np.arange(0, len(y_test)):
  print("Предсказанное: ", pred[i][0], ", истинное: ", y_test[i], ", разность: ", diff[i])

Предсказанное:  1530.4155 , истинное:  1550.79 , разность:  20.374472656249964
Предсказанное:  1046.0164 , истинное:  1042.49 , разность:  3.526357421874991
Предсказанное:  5408.048 , истинное:  5415.68 , разность:  7.632148437500291
Предсказанное:  3972.8057 , истинное:  4002.5 , разность:  29.6943359375
Предсказанное:  1446.4753 , истинное:  1457.16 , разность:  10.684658203125082
Предсказанное:  3952.1636 , истинное:  3964.57 , разность:  12.406425781250164
Предсказанное:  3973.687 , истинное:  3975.25 , разность:  1.56298828125
Предсказанное:  962.9909 , истинное:  964.82 , разность:  1.8290942382813
Предсказанное:  1027.2756 , истинное:  1032.27 , разность:  4.994365234374982
Предсказанное:  602.94806 , истинное:  573.83 , разность:  29.11805908203121
Предсказанное:  5396.4404 , истинное:  5384.47 , разность:  11.970429687499745
Предсказанное:  1371.598 , истинное:  1412.02 , разность:  40.42197753906248
Предсказанное:  1643.4089 , истинное:  1671.45 , разность:  28.04106445312504

KeyboardInterrupt: ignored

In [ ]:
#for i,j in zip(pred, y_test):
#  print("Предсказанное:", i, ", истинное:", j)

In [ ]:
vec = np.array([])
for i in np.arange(0, len(pred)):
  vec = np.append(vec, pred[i][0])
vec

array([1530.41552734, 1046.01635742, 5408.04785156, ..., 3616.3762207 ,
       1504.60070801,  822.32598877])

In [ ]:
# Коэффициент корреляции
СС_tuner = np.corrcoef(vec, y_test)
СС_tuner = СС_tuner[0][1]
print(f'Коэффициент корреляции с истинными данными: {СС_tuner}')

Коэффициент корреляции с истинными данными: 0.9993158444310267
